In [13]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Copy contents from My Drive to "/content" in order to import all scripts.
!cp -r /content/drive/My\ Drive/SC_RL /content

**Goal of this Project:** Maintain inventory of all products while minimizing operational costs.

In [14]:
## Import libraries.
import numpy as np
import os

from pathlib import Path

from SC_RL.Environments import warehouse_store

In [ ]:
def reward_function(states, actions):
  '''
  Should ensure that inventory is stocked, but at the same time, ensure that
  wastage is minimized.
  1 - (products with no inventory left/total no. of products)
    - (quantity of expired products/total number of products)
  '''
  reward = 1
  return reward

In [24]:
metadata_file = Path(os.getcwd()+"/SC_RL/data/instacart-market-basket-analysis/products_metadata.xlsx")
forecast_data = Path(os.getcwd()+"/SC_RL/data/instacart-market-basket-analysis/scenarios.xlsx")
w = warehouse_store.warehouse_store()
w.initialize(metadata_file,forecast_data)